In [11]:
!pip install pyaudio
!pip install SpeechRecognition
!pip install openai opencv-python gtts playsound pyttsx3 numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-11.0-arm64-cpython-311
      creating build/lib.macosx-11.0-arm64-cpython-311/pyaudio
      copying src/pyaudio/__init__.py -> build/lib.macosx-11.0-arm64-cpython-311/pyaudio
      running build_ext
      building 'pyaudio._portaudio' extension
      creating build/temp.macosx-11.0-arm64-cpython-311
      creating build/temp.macosx-11.0-arm64-cpython-311/src
      creating build/temp.macosx-11.0-arm64-cpython-311/src/pyaudio
      clang -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /Users/karimghazz

In [3]:
!pip install openai opencv-python mediapipe gtts playsound SpeechRecognition


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.2
    Uninstalling protobuf-4.25.2:
      Successfully uninstalled protobuf-4.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.1 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.


In [3]:
import cv2
import openai
from gtts import gTTS
from playsound import playsound
from ultralytics import YOLO
import speech_recognition as sr

# Initialize OpenAI API
openai.api_key = 'sk-proj-fFUpG5eVab924oEJinyVT3BlbkFJgLjTZyIQHBXVpmtP749I'

# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # You can replace with the path to your YOLOv8 model

# COCO class names
class_names = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train',
    'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter',
    'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
    'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
    'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'TV', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
    'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'coffee machine', 'cat', 'coffee cup', 'sandwich', 'bread',
    'eating', 'woman'
]

# Function to generate description using OpenAI
def generate_description(objects):
    prompt = f"I see the following objects: {', '.join(objects)}. Can you describe this scene?"
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Use GPT-4 model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message['content'].strip()

# Initialize speech recognition
recognizer = sr.Recognizer()
microphone = sr.Microphone()

# Function to recognize speech
def recognize_speech():
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        print("Listening...")
        audio = recognizer.listen(source)
    try:
        return recognizer.recognize_google(audio)
    except sr.UnknownValueError:
        return "Sorry, I did not understand that."
    except sr.RequestError:
        return "Could not request results; check your network connection."

# Start capturing video from the phone camera
cap = cv2.VideoCapture(0)  # Replace with your phone's IP camera URL

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame for YOLOv8
    results = model(frame)

    # Extract object labels from YOLOv8 results
    objects = []
    for result in results:
        if hasattr(result, 'boxes') and result.boxes:
            for box in result.boxes:
                class_id = int(box.cls)
                objects.append(class_names[class_id])
    
    # Generate description
    if objects:
        description = generate_description(objects)
        print(description)

        # Convert text to speech using gTTS
        tts = gTTS(description)
        tts.save("description.mp3")
        try:
            playsound("description.mp3", True)
        except Exception as e:
            print(f"Error playing sound: {e}")

    # Display the video with YOLOv8 detections
    annotated_frame = results[0].plot()  # Access the first result for rendering
    cv2.imshow('Video', annotated_frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 3 bottles, 1 vase, 52.7ms
Speed: 2.2ms preprocess, 52.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
In this scene, there are three bottles, possibly of varying shapes, sizes, or colors. The bottles could be set up for a range of purposes: holding liquids such as water or juice, for decorative purposes, or perhaps they are antiques with their own histories. Alongside the bottles, there's also a vase. This could be a design statement in the scene - perhaps it's tall, short, slender, or wide, made of glass, ceramic or another material. It might be empty, waiting to hold a bouquet of flowers, or already hosting an arrangement. The bottles and vase could form a collection or display, carefully curated or casually collected, creating an interesting mix of vessels in this setting.

0: 384x640 1 person, 51.6ms
Speed: 15.1ms preprocess, 51.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
As an AI, I don't have direct perception to see objec

KeyboardInterrupt: 